#### Questão 4 - P2 Transferêcia de Calor 2020

Nome: Alessandro Melo de Oliveira  \
Nº USP: 10788662

In [1]:
#Bibliotecas usadas
import pandas as pd
import numpy as np

In [2]:
## Propriedades do ar, colhidas no Engineering ToolBox
air_data = pd.read_csv('air_dry_properties.txt', delim_whitespace=True)
air_data['Thermal_Conductivity'] = air_data['Thermal_Conductivity'].apply(lambda x: x* 10**-2)
air_data['Kinematic_Viscosity'] = air_data['Kinematic_Viscosity'].apply(lambda x: x* 10**-5)
air_data['Thermal_Diffusivity'] = air_data['Thermal_Diffusivity'].apply(lambda x: x*10**-6)
air_data['Cp'] = air_data['Cp'].apply(lambda x: x*10**3)
air_data['Cv'] = air_data['Cv'].apply(lambda x: x*10**3)

air_data['alpha'] = [k/(rho*cp) for k,rho,cp in zip(air_data['Thermal_Conductivity'], air_data['Density'],
                                       air_data['Cp'])]
air_data['beta'] = [1/i for i in air_data['Temperature'] ]

#Interpolação dos dados 
def air(t_new):
    
    k_new = np.interp(t_new,air_data['Temperature'],air_data['Thermal_Conductivity'])  
    alpha_new= np.interp(t_new,air_data['Temperature'],air_data['alpha'])  
    pran_new= np.interp(t_new,air_data['Temperature'],air_data['Prandtl_Number'])  
    beta_new = np.interp(t_new,air_data['Temperature'],air_data['beta'])  
    v_new = np.interp(t_new,air_data['Temperature'],air_data['Kinematic_Viscosity'])  
    
    return k_new, alpha_new, pran_new, beta_new, v_new

In [3]:
#Dados do problema
A = 0.7**2 # [m^2]
e = 30e-3 # [m]
T_upper = 25 + 273.15 # [K]
T_lower = 150 + 273.15 # [K]
k = 0.8 # [W/m^2.K]
L = A/(0.7*4) # [m]
g = 9.81 # [m/s^2]

In [5]:
## Definitivamente esse não é o jeito mais rápido de se resolver, mas o semestre ta corrido e foi o único jeito que eu sabia.
## Creio ter como fazer utilizando algum solver do scipy.

data = pd.DataFrame(columns = ['T_upper ºC','T_lower ºC', 'Q_upper','Q_lower','Q_middle', 'Q_dot', 'h_upper','h_lower','criterio'])

chutes = np.linspace(T_upper+55, T_lower-55,500)

for T_up in chutes:
    for T_low in chutes:
        
        Tf_up = (T_up + T_upper)/2
        Tf_low = (T_low + T_lower)/2
        
        #Número de Rayleigh
        ray_up = (g*air(Tf_up)[3]*(abs(T_up-T_upper))*(L**3))/(air(Tf_up)[1] * air(Tf_up)[4])
        ray_low = (g*air(Tf_low)[3]*(abs(T_low-T_lower))*(L**3))/(air(Tf_low)[1] * air(Tf_low)[4])
        
        
        #Número de Nusselt
        if 10e4 <= ray_up <= 10e7:
            Nu_up = 0.54 * (ray_up)**(1/4)
        elif 10e7 < ray_up <= 10e11:
            Nu_up = 0.15 * (ray_up)**(1/3)
            
        if 10e4 <= ray_low <= 10e7:
            Nu_low = 0.54 * (ray_low)**(1/4)
        elif 10e7 < ray_low <= 10e11:
            Nu_low = 0.15 * (ray_low)**(1/3)
        
        #Coeficientes de película    
        h_upper = (Nu_up*air(Tf_up)[0])/L
        h_lower = (Nu_low*air(Tf_low)[0])/L
        
        #Resistências térmicas
        R1 = 1/(h_lower*A)
        R2 = e/(k*A)
        R3 = 1/(h_upper*A)
        Req = R1 + R2 + R3
        
        #Taxas de transferência de calor
        Q_dot = (T_lower-T_upper)/Req
        Q_upper = (T_up - T_upper)/R3
        Q_lower = (T_lower - T_low)/R1
        Q_middle = (T_low - T_up)/R2
        
        #Critério para averiguar a condição de contorno
        a = abs(Q_upper - Q_lower)
        b = abs(Q_upper - Q_middle)
        c = abs(Q_lower-Q_middle)
        criterio = a+b+c
        
        row = [T_up - 273.15, T_low -273.15, Q_upper, Q_lower, Q_middle, Q_dot, h_upper, h_lower, criterio]
        data.loc[len(data)] = row
        
get_min = data['criterio'].min()
data[data['criterio']==get_min]

,T_upper ºC,T_lower ºC,Q_upper,Q_lower,Q_middle,Q_dot,h_upper,h_lower,criterio
5908,80.330661,92.264529,155.881078,155.786231,155.935872,155.842482,5.749518,5.506686,0.299281
